# Gravity

In [2]:
from scenario import *
import pybullet as p
from time import sleep
p.connect(p.DIRECT)
boxid=p.createCollisionShape(p.GEOM_BOX,halfExtents=[0.5,0.5,0.5])
p.createMultiBody(baseMass=0,
                    baseCollisionShapeIndex=boxId,
                    basePosition=[0, 0, -0.1])
print(boxid)
scen=Scenario()
cube=Cube()
cube.position=Vector3(0,0,2)
cube.mass=1

scen.add(cube)
while scen.t<3:
    scen.step(0.1)
    scen.render()
    sleep(0.1)

0


# Collision Detection

## Get normal vector of a plane defined by two vectors

In [1]:
from scenario import *
v1=Vector3(0,1,2)
v2=Vector3(3,4,5)
n=v1.cross(v2)
n

tensor([-3.,  6., -3.])

In [3]:
l1=Vector(v1)
l1.id="v1"
l1.color=Color(r=1)
l2=Vector(v2)
l2.color=Color(g=1)
l3=Vector(n)
l3.color=Color(b=1)
Scenario(l1,l2,l3).render()

## Line intersect normal-point plane
Assume normal vector $\textbf{n}$ and point $\textbf{p}_0$ defines a plane, $\textbf{p}$ represents any point on this plane. Then

$\textbf{n}\cdot(\textbf{p}-\textbf{p}_0)=0$

Assume two points $\textbf{a}$ and $\textbf{b}$ defines a line, $\textbf{p}$ is any point on this line, and t is a scalar. Then

$\textbf{p}=\textbf{a}+t(\textbf{b}-\textbf{a})$

If the line and plane have an intersection point $\textbf{p}_i$. Then

$\textbf{n}\cdot(\textbf{p}_i-\textbf{p}_0)=0$

$\textbf{p}_i=\textbf{a}+t(\textbf{b}-\textbf{a})$

$\Rightarrow \textbf{n}\cdot[\textbf{a}+t(\textbf{b}-\textbf{a})-\textbf{p}_0]=0$

$\Rightarrow \textbf{n}\cdot\textbf{a}+t\textbf{n}\cdot(\textbf{b}-\textbf{a})-\textbf{n}\cdot\textbf{p}_0=0$

$\Rightarrow t=\frac{\textbf{n}\cdot(\textbf{p}_0-\textbf{a})}{\textbf{n}\cdot(\textbf{b}-\textbf{a})}$

$\Rightarrow \textbf{p}_i=\textbf{a}+\frac{\textbf{n}\cdot(\textbf{p}_0-\textbf{a})}{\textbf{n}\cdot(\textbf{b}-\textbf{a})}(\textbf{b}-\textbf{a})$

In [4]:
from scenario import  *
n=Vector3(1,1,1)
p0=Vector3(5,6,2)
a=Vector3(1,3,5)
b=Vector3(2,-2,-1)
t=n.dot(p0-a)/n.dot(b-a)
pi=a+t*(b-a)
print(t)
Scenario(Plane(p0,n),Point(pi),Vector(a,b)).render()

tensor(-0.4000)


## Line intersect triangle

Three points $A,B,C$ define a triangle $\triangle ABC$.

Another two points $P_0,P_1$ define a line segment.

If there is an intersection $P_x$ between the triangle and the line segment, then

$u\overrightarrow{CA}+v\overrightarrow{CB}+C=P_x$ and $u+v<=1$

$P_x=P_0+t\overrightarrow{P_0P_1}$

$\Rightarrow u\overrightarrow{CA}+v\overrightarrow{CB}+C = P_0+t\overrightarrow{P_0P_1}$

$\Rightarrow t\overrightarrow{P_0P_1}+ u\overrightarrow{AC}+v\overrightarrow{BC}= C-P_0$

$\Rightarrow [\overrightarrow{P_0P_1},\overrightarrow{AC},\overrightarrow{BC}]\left[ \begin{array} \\ t \\ u \\ v\end{array}\right]= C-P_0$

$\Rightarrow \left[ \begin{array} \\ t \\ u \\ v\end{array}\right]= [P_1-P_0,C-A,C-B]^{-1}(C-P_0)$

In [74]:
from scenario import *
A=Vector3(1,2,0)
B=Vector3(3,6,1)
C=Vector3(5,-2,0)
P0=Vector3(3,3,3)
P1=Vector3(2,4,-3)
t,u,v=torch.cat(((P1-P0).unsqueeze(1),(C-A).unsqueeze(1),(C-B).unsqueeze(1)),dim=1).inverse().mv(C-P0)
print(t,u,v)
Px=P0+t*(P1-P0)
Scenario(Vector(C,A),Vector(C,B),Vector(P0,P1),Point(Px)).render()

tensor(0.4167) tensor(0.3542) tensor(0.5000)


## Collision between a line segment and a cube

Assume $P$ is a point within a cube and it is represented in the cube's local frame. The cube's has three direction vectors $\textbf{d}_x,\textbf{d}_y,\textbf{d}_z$, which size are $s_x,s_y,s_z$. Then

$P=(us_x,vs_y,ws_z),u \leqslant1,v \leqslant1,w \leqslant1$

In [99]:
cube=Cube(4,2,1)
cube.rotation=Rotation.Eular(0.1,-0.4,1)
p=Vector3(0.2,0.9,-0.4)/2
P=cube.position+cube.rotation*(cube.scale*p)
Scenario(cube,Point(P)).render()

## Collision between two cubes

In [100]:
from scenario import *
red=Cube(4,1.8,1.48)
red.color=Color(r=1)
blue=Cube()
blue.color=Color(b=1)
red.position=Vector3(1,2,3)
scen=Scenario()
scen.add(red,blue)
scen.render()
def onkey(key):
    if key=='q':
        red.position+=Vector3(0.1,0,0)
    elif key=='a':
        red.position+=Vector3(-0.1,0,0)
    elif key=='w':
        red.position+=Vector3(0,0.1,0)
    elif key=='s':
        red.position+=Vector3(0,-0.1,0)
    elif key=='e':
        red.position+=Vector3(0,0,0.1)
    elif key=='d':
        red.position+=Vector3(0,0,-0.1)
    elif key=='r':
        red.rotation*=Rotation.Eular(z=0.1)
    elif key=='f':
        red.rotation*=Rotation.Eular(z=-0.1)
    scen.render()
scen.on_key=onkey

In [102]:
rx=red.rotation*(red.scale*Vector3(1,0,0))+red.position
ry=red.rotation*(red.scale*Vector3(0,1,0))+red.position
rz=red.rotation*(red.scale*Vector3(0,0,1))+red.position
bx=blue.rotation*(blue.scale*Vector3(1,0,0))+blue.position
by=blue.rotation*(blue.scale*Vector3(0,1,0))+blue.position
bz=blue.rotation*(blue.scale*Vector3(0,0,1))+blue.position
Tr=torch.cat((rx.unsqueeze(1),ry.unsqueeze(1),rz.unsqueeze(1)),dim=1)
Tb=torch.cat((bx.unsqueeze(1),by.unsqueeze(1),bz.unsqueeze(1)),dim=1)
Tr.inverse()*Tb

tensor([[0.2358, -0.0000, -0.0000],
        [-0.0000, 0.4149, -0.0000],
        [-0.0000, -0.0000, 0.3636]])